# Introduction

In this tutorial we will learn about different environments, then implement policies, dynamic programming algorithms and temporal difference algorithms to study their performance.

### N-armed bandit

In an N-armed bandit task, there is only a single state. Our implementation is a 4-armed bandit, which means there are 4 available actions, and each one returns a different reward.

<img src="https://github.com/goptavares/intro-to-rl/blob/master/fig/4ArmedBandit.png?raw=true" width=500 height=225>

### Cliff world

In this 4x10 grid there are 40 states and 4 possible actions: right, up, left and down. Falling into the cliff incurs a negative reward of -100 and ends the episode; moving into any other state incurs a reward of -1; moving into the world borders stays in the same place; moving anywhere from the goal state (state G in the figure) ends the episode.

<img src="https://github.com/goptavares/intro-to-rl/blob/master/fig/CliffWorld.png?raw=true" width=600 height=280>

### Quentin's world

In this 10x10 grid there are 100 states and 4 possible actions: right, up, left and down. The start state is in green in the figure; moving into one of the red states incurs a reward of -1; moving into the world borders stays in the same place; moving into the goal state (yellow square in the upper right corner) gives you a reward of 1; and moving anywhere from the goal state ends the episode.

<img src="https://github.com/goptavares/intro-to-rl/blob/master/fig/QuentinsWorld.png?raw=true" width=300 height=300>

### Multi-room windy gridworld with cliffs

In this 12x14 grid there are 168 states and 4 possible actions: right, up, left and down. The start state is marked with an S in the figure, and there are 2 goals states marked with a G. Each goal state is inside a room (the room walls are marked with darker lines). Moving into a goal states gives you a reward of 100. Moving into a wall or outside the world borders stays in the same place. The two rooms are windy, which means the resultant next states inside the rooms are shifted; the wind direction is indicated by a blue arrow, and the wind strength (size of the shift) in each column is indicated by a number between 0 and 2. There are also two cliffs marked in gray; falling into a cliff incurs a reward of -100 and ends the episode.

<img src="https://github.com/goptavares/intro-to-rl/blob/master/fig/gridworld.png?raw=true" width=451.2 height=447.2>


# Notebook setup

In [0]:
% matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
from pylab import *

!git clone https://github.com/goptavares/intro-to-rl.git
%run intro-to-rl/RL_worlds.py
%run intro-to-rl/plot_util.py

# Helper functions

In [0]:
def default_params(environment):
  """
  Define the default parameters.
  Args:
    environment: an object corresponding to the environment.
  Returns:
    a dictionary containing the default parameters, where the keys
        are strings (parameter names).
  """
  params = dict()
  params['environment'] = environment
  
  params['alpha'] = 0.1  # learning rate
  params['beta'] = 10  # inverse temperature
  params['epsilon'] = 0.05  # epsilon-greedy policy
  params['epsilon_decay'] = 0.9
  params['gamma'] = 1.0  # no discounting

  params['policy'] = 'epsilon_greedy'
  params['learning_rule'] = 'q_learning'

  if environment.name == 'n_armed_bandit':
    params['gamma'] = 0.9
  elif environment.name == 'cliff_world':
    params['gamma'] = 1.0  # no discounting
  elif environment.name == 'quentins_world':
    params['gamma'] = 0.9
  elif environment.name == 'windy_cliff_grid':
    params['gamma'] = 0.8

  return params

In [0]:
def init_state(params):
  """
  Initialize the state at the beginning of an episode.
  Args:
    params: a dictionary containing the default parameters.
  Returns:
    an integer corresponding to the initial state.
  """
  if params['environment'].name == 'windy_cliff_grid':
    return 0
  elif params['environment'].name == 'n_armed_bandit':
    return 0
  elif params['environment'].name == 'cliff_world':
    return 0
  elif params['environment'].name == 'quentins_world':
    return 54

In [0]:
def update_state(state, action, params):
  """
  State transition based on world, action and current state.
  Args:
    state: integer corresponding to the current state.
    action: integer corresponding to the action taken.
    params: a dictionary containing the default parameters.
  Returns:
    an integer corresponding to the next state;
    an integer corresponding to the reward received.
  """
  next_state, reward = params['environment'].get_outcome(state, action)
  return next_state, reward

In [0]:
def call_policy(state, value, params):
  """
  Call a policy to choose actions, given current state and value function.
  Args:
    state: integer corresponding to the current state.
    value: a matrix indexed by state and action.
    params: a dictionary containing the default parameters.
  Returns:
    an integer corresponding action chosen according to the policy.
  """
  if params['policy'] == 'epsilon_greedy':
    return epsilon_greedy(state, value, params)
  elif params['policy'] == 'softmax':
    return softmax(state, value, params)
  else:
    # Random policy (if policy not recognized, choose randomly).
    return randint(params['environment'].n_actions)

In [0]:
def update_value(prev_state, action, reward, state, value, params):
  """
  Update the value function.
  Args:
    prev_state: an integer corresponding to the previous state.
    action: an integer correspoding to action taken.
    reward: a float corresponding to the reward received.
    state: an integer corresponding to the current state;
        should be None if the episode ended.
    value: a matrix indexed by state and action.
    params: a dictionary containing the default parameters. 
  Returns:
    the updated value function (matrix indexed by state and action).
  """
  if params['learning_rule'] == 'q_learning':
    # Off policy learning.
    return q_learning(prev_state, action, reward, state, value, params)
  elif params['learning_rule'] == 'sarsa':
    # On policy learning.
    return sarsa(prev_state, action, reward, state, value, params)
  else:
    print('Learning rule not recognized')

In [0]:
def run_learning(value, params, n_episodes, max_steps):
  """
  Args:
    value: a matrix indexed by state and action.
    params: a dictionary containing the default parameters.
    n_episodes: integer, number of episodes to run.
    max_steps: integer, maximum number of steps to take in each episode.
  Returns:
    a dictionary where the keys are integers (episode numbers)
        and the values are integers (total rewards per episode);
    the updated value function (matrix indexed by state and action).
  """
  reward_sums = np.zeros(n_episodes)

  # Loop over episodes.
  for episode in range(n_episodes):
    # Initialize state.
    state = init_state(params)    
    step = 0
    reward_sum = 0

    # Make sure to break after max number of steps.
    while step < max_steps:
      # Get action from policy.
      action = call_policy(state, value, params)  
      # Update state based on action.
      next_state, reward = update_state(state, action, params)
      # Update value function.
      value = update_value(state, action, reward, next_state, value, params) 
      state = next_state
      # Sum the rewards obtained.
      reward_sum += reward  
      step += 1
      if next_state == None:
        # Episode ends.
        break  
    reward_sums[episode] = reward_sum
  return reward_sums, value

# Exercise 1: Policy Evaluation

**Complete the policy evaluation algorithm below.**

The function takes the policy to be evaluated, the default parameters and a threshold for the stopping criterion as input and returns the value function.


In [0]:
def policy_evaluation(policy, params, theta=0.000001):
  """
  Evaluates a policy given an environment and a full description of the
      environment's dynamics.
  Args:
    policy: a matrix indexed by state and action, where each entry is a float
        corresponding to the probability of taking that action from that state.
    params: a dictionary containing the default parameters.
    theta: float; we stop evaluation once our value function change is less than
        theta for all states.
  Returns:
    the value function (vector indexed by state).
  """
  env = params['environment']
  outcomes = env.get_all_outcomes()
  
  # Start with a random (all zeros) value function.
  value = np.zeros(env.n_states)

  while True:
    delta = 0
    # For each state, perform a "full backup".
    for state in range(env.n_states):
      v = 0
      # Look at all possible next actions.
      for action, action_prob in enumerate(policy[state]):
        # For each action, look at all possible next states.
        for prob, next_state, reward in outcomes[state, action]:
          if next_state == None:
              value_next = 0
          else:
              value_next = value[next_state]
          # Calculate the expected value.
          v += action_prob * prob * (reward + params['gamma'] * value_next)
      # How much our value function changed (across any states).
      delta = max(delta, np.abs(value[state] - v))
      value[state] = v
    # Stop evaluating once our value function change is below the threshold.
    if delta < theta:
        break
  return value

# Exercise 2: Policy Iteration

**Complete the policy iteration algorithm below.**

The function takes the default parameters and a threshold for the stopping criterion as input and returns the optimal policy and the value function for the optimal police. It uses the policy evaluation function written in Exercise 1.

In [0]:
def policy_iteration(params, theta=0.000001):
  """
  Iteratively evaluates and improves a policy until an optimal policy is found.
  Args:
    params: a dictionary containing the default parameters.
    theta: float, to be used in policy_evaluation; we stop evaluation once our
        value function change is less than theta for all states.
  Returns:
    the optimal policy (matrix indexed by state and action);
    the corresponding value function (vector indexed by state).
  """
  env = params['environment']
  outcomes = env.get_all_outcomes()

  # Start with a random policy.
  policy = np.ones([env.n_states, env.n_actions]) / env.n_actions
  
  while True:
    # Evaluate the current policy.
    value = policy_evaluation(policy, params, theta)

    # This will be set to false if we make any changes to the policy.
    policy_stable = True
    
    # Iterate over all states.
    for state in range(env.n_states):
      # The best action we would take under the current policy.
      chosen_action = np.argmax(policy[state])
      
      # Find the best action by one-step lookahead using the new value function.
      # Ties are resolved arbitarily.
      action_values = np.zeros(env.n_actions)
      for action in range(env.n_actions):
        for prob, next_state, reward in outcomes[state, action]:
          if next_state == None:
            value_next = 0
          else:
            value_next = value[next_state]
          action_values[action] += prob * (reward + params['gamma'] * value_next)
      best_action = np.argmax(action_values)
      
      # Greedily update the policy.
      if chosen_action != best_action:
          policy_stable = False
      policy[state] = np.eye(env.n_actions)[best_action]

    # If the policy is stable, we've found an optimal policy.
    if policy_stable:
        break
  return policy, value

# Exercise 3: Value Iteration

One drawback to policy iteration is that each of its iterations involves policy evaluation, which may be slow due to multiple sweeps through the state set. We can make this process faster by truncating the policy evaluation step of policy iteration, stopping it after just one sweep (one back-up of each state). This modified algorithm is called value iteration.

**Complete the value iteration algorithm.**

The function takes the default parameters and a threshold for the stopping criterion as input and returns the policy and corresponding value function. There is also a helper function to calculate the one-step lookahead, i.e., the value for all actions in a given state using the current estimate of the value function.

In [0]:
def one_step_lookahead(state, value, params):
  """
  Helper function to calculate the value for all actions in a given state.
  Args:
    state: int, the state to consider.
    value: the value function to use as an estimator (vector indexed by state).
    params: a dictionary containing the default parameters.
  Returns:
    a vector indexed by action containing the expected value of each action.
  """
  env = params['environment']
  outcomes = env.get_all_outcomes()
  A = np.zeros(env.n_actions)
  for action in range(env.n_actions):
    for prob, next_state, reward in outcomes[state, action]:
      if next_state == None:
        value_next = 0
      else:
        value_next = value[next_state]
      A[action] += prob * (reward + params['gamma'] * value_next)
  return A

def value_iteration(params, theta=0.000001):
  """
  Value iteration algorithm.
  Args:
    params: a dictionary containing the default parameters.
    theta: float; we stop evaluation once our value function change is less
          than theta for all states.
  Returns:
    the optimal policy (matrix indexed by state and action);
    the corresponding value function (vector indexed by state).
  """    
  env = params['environment']
  value = np.zeros(env.n_states)

  while True:
    # Stopping condition.
    delta = 0
    # Update each state.
    for state in range(env.n_states):
      # Do a one-step lookahead to find the best action.
      A = one_step_lookahead(state, value, params)
      best_action_value = np.max(A)
      # Calculate delta across all states seen so far.
      delta = max(delta, np.abs(best_action_value - value[state]))
      # Update the value function.
      value[state] = best_action_value        
    # Check if we can stop.
    if delta < theta:
      break
  
  # Create a deterministic policy using the optimal value function.
  policy = np.zeros([env.n_states, env.n_actions])
  for state in range(env.n_states):
    # One step lookahead to find the best action for this state.
    A = one_step_lookahead(state, value, params)
    best_action = np.argmax(A)
    # Always take the best action.
    policy[state, best_action] = 1.0

  return policy, value

# Exercise 4

The code below allows you to test the performance of policy iteration and value iteration for a selected world (try cliff world, Quentin's world or windy cliff grid). Use the functions provided in the plot_util module to:

* Plot the action corresponding to the policy at each state;
* Plot the value associated with each state.

**A.** Experiment with different parameter values.

* Pick a range for the temporal discount factor  γ  and look at how the results change.

**B.** To make sure that your algorithms have been implemented correctly, compare your results to the ones shown below.

**C.** Compare the results obtained using policy iteration vs. value iteration. What differences do you notice between the two approaches?

###Cliff world using policy improvement and  γ =0.9:

<img src="https://github.com/goptavares/intro-to-rl/blob/master/fig/policyimp_actions.png?raw=true" height=200 width=320>

<img src="https://github.com/goptavares/intro-to-rl/blob/master/fig/policyimp_maxval.png?raw=true" height=200 width=320>


###Quentin's world using using value iteration and  γ =0.9:

<img src="https://github.com/goptavares/intro-to-rl/blob/master/fig/valueit_actions.png?raw=true" height=250 width=250>

<img src="https://github.com/goptavares/intro-to-rl/blob/master/fig/valueit_maxval.png?raw=true" height=250 width=280>

In [0]:
# Choose a world.
env = cliff_world()
# env = quentins_world()
# env = windy_cliff_grid()

params = default_params(environment=env)

# Decision-maker: choose parameter values.
params['gamma'] = 0.9

# Choose a DP algorithm: policy iteration or value iteration.
policy, value = policy_iteration(params)
# policy, value = value_iteration(params)

fig = plot_quiver_max_action(env, policy)
fig = plot_heatmap_max_val(env, value)

# Quick refresher

### Learning algorithms and policies

**Learning algorithms:**

*Sarsa (on-policy)*

\begin{align}
Q(s_t,a_t) \leftarrow Q(s_t,a_t) + \alpha \big(r_t + \gamma Q(s_{t+1},a_{t+1}) - Q(s_t,a_t)\big)
\end{align}

with temporal discount rate $\gamma$ and learning rate $\alpha$.

*Q-learning (off-policy)*

\begin{align}
Q(s_t,a_t) \leftarrow Q(s_t,a_t) + \alpha \big(r_t + \gamma\max_\limits{a} Q(s_{t+1},a_{t+1}) - Q(s_t,a_t)\big)
\end{align}


**Policies:**

*Epsilon-greedy*

\begin{align}
P(a_t|s_t) = \epsilon \frac{1}{N_a}  + (1-\epsilon)1[a_t =\max_\limits{a}Q(a_t,s_t)]
\end{align}

*Softmax*

\begin{align}
P(a_t|s_t) = \frac{\exp(Q(a_t,s_t)/\tau)}{\Sigma_{i=1}^n \exp(Q(i)/\tau)}
\end{align}

# Exercise 5: Decision Policies

**A.** Complete the epsilon-greedy policy function below.

**B.** [Optional] Complete the softmax policy function below.

Both functions take the current state, the value function and default parameters as input and return an action.

In [0]:
def epsilon_greedy(state, value, params):
  """
  Epsilon-greedy policy: selects the maximum value action with probability
      (1-epsilon) and selects randomly with epsilon probability.
  Args:
    state: an integer corresponding to the current state.
    value: a matrix indexed by state and action.
    params: a dictionary containing the default parameters. 
  Returns:
    action: an integer corresponding action chosen according to the policy.
  """
  value_now = value[state,:]
  if rand() > params['epsilon']:
    action = where(value_now == max(value_now))[0][0]
  else:
    action = randint(len(value_now))
  return action

In [0]:
def softmax(state, value, params):
  """
  Softmax policy: selects action probabilistically depending on the value.
  Args:
    state: an integer corresponding to the current state.
    value: a matrix indexed by state and action.
    params: a dictionary containing the default parameters.
  Returns:
    an integer corresponding to the action chosen according to the policy.
  """
  value_now = value[state,:]
  # Use params['beta'] (the inverse temperature) which is equal to 1/tau.
  prob = exp(value_now * params['beta'])
  prob = prob / sum(prob)  # Normalize
  cum_prob = cumsum(prob)  # Cumulative sum
  action = where(cum_prob > rand())[0][0]
  return action

# Exercise 6: Learning Algorithms

**A.** Complete the Q-learning (off-policy) algorithm below.

**B.** Modify the Q-learning algorithm to obtain a Sarsa (on-policy) algorithm.

Both functions take the previous state, action taken, reward received, value function, current state and default parameters and return the updated value function.

In [0]:
def q_learning(prev_state, action, reward, state, value, params):
  """
  Q-learning: updates the value function and returns it.
  Args:
    prev_state: an integer corresponding to the previous state.
    action: an integer corresponding to the action taken.
    reward: a float corresponding to the reward received.
    state: an integer corresponding to the current state.
    value: a matrix indexed by state and action.
    params: a dictionary containing the default parameters. 
  Returns:
    the updated value function (matrix indexed by state and action).
  """
  # Maximum value at current state.
  if state == None:
    max_value = 0
  else:
    max_value = max(value[state,:])
  
  # Value of previous state-action pair.
  prev_value = value[prev_state, action]
  # Reward prediction error. Use params['gamma'], the temporal discount factor.
  delta = reward + params['gamma'] * max_value - prev_value  
  
  # Update value of previous state-action pair. Use params['alpha'], the
  # learning rate.
  value[prev_state, action] = prev_value + params['alpha'] * delta
  
  return value

In [0]:
def sarsa(prev_state, action, reward, state, value, params):
  """
  Sarsa: updates the value function and returns it.
  Args:
    prev_state: an integer corresponding to the previous state.
    action: an integer correspoding to action taken.
    reward: a float corresponding to the reward received.
    state: an integer corresponding to the current state.
    value: a matrix indexed by state and action.
    params: a dictionary containing the default parameters.
  Returns:
    the updated value function (matrix indexed by state and action).
  """
  # Select the expected value at current state based on our policy by sampling
  # from it.
  if state == None:
    policy_value = 0
  else:
    # Sample action from the policy for the next state.
    policy_action = call_policy(state, value, params)
    # Get the value based on the action sampled from the policy.
    policy_value = value[state, policy_action]
  
  # Value of previous state-action pair.
  prev_value = value[prev_state, action]
  # Reward prediction error. Use params['gamma'], the temporal discount factor.
  delta = reward + params['gamma'] * policy_value - prev_value
  
  # Update value of previous state-action pair. Use params['alpha'], the
  # learning rate.
  value[prev_state, action] = prev_value + params['alpha'] * delta
  
  return value

# Exercise 7

The code below allows you to select a world, a learning algorithm and a decision policy.

**A.** Run 500 episodes (visits to the world) with learning across episodes. Make sure to set a maximum number of steps per episode (e.g. 1000). Use the functions provided in the plot_util module to:
* Plot the value associated with each action at each state;
* Plot the action corresponding to the maximum value at each state;
* Plot the maximum value in each state;
* Plot the total reward obtained in each episode.

**B.** To make sure that your algorithms have been implemented correctly, compare your results to the ones shown below.

**C.** Experiment with different values for the parameters:
* Pick a range for the learning rate $\alpha$ and look at how the results change.
* Pick a range for the inverse temperature $\beta$ (using a softmax policy) and look at how the results change.
* Pick a range for $\epsilon$ (using an $\epsilon$-greedy policy) and look at how the results change.
* Pick a range for the temporal discount factor $\gamma$ and look at how the results change.

**D.** Explore the cliff world with an $\epsilon$-greedy policy (try $\epsilon$=0.1) comparing the performance of Q-learning (off-policy) and Sarsa (on-policy). What differences do you notice? What do these differences tell us about on- and off-policy learning?

**E.** Compare all of your results with those obtained with the Dynamic Programming algorithms in Exercise 4.

* What do you notice about the differences in performance between Dynamic Programming and TD learning algorithms?
* What are some of the advantages and disadvantages of each approach?

### Cliff world using Q-learning and an $\epsilon$-greedy policy with $\epsilon$=0.1 and $\alpha$=0.3:

<img src="https://github.com/goptavares/intro-to-rl/blob/master/fig/qlearning_values.png?raw=true" height=200 width=320>

<img src="https://github.com/goptavares/intro-to-rl/blob/master/fig/qlearning_actions.png?raw=true" height=200 width=320>

<img src="https://github.com/goptavares/intro-to-rl/blob/master/fig/qlearning_maxval.png?raw=true" height=200 width=320>

<img src="https://github.com/goptavares/intro-to-rl/blob/master/fig/qlearning_rewards.png?raw=true" height=200 width=320>

### Quentin's world using Sarsa and a softmax policy with $\beta$=10 and $\alpha$=0.4:

<img src="https://github.com/goptavares/intro-to-rl/blob/master/fig/sarsa_values.png?raw=true" height=200 width=320>

<img src="https://github.com/goptavares/intro-to-rl/blob/master/fig/sarsa_actions.png?raw=true" height=200 width=320>

<img src="https://github.com/goptavares/intro-to-rl/blob/master/fig/sarsa_maxval.png?raw=true" height=200 width=320>

<img src="https://github.com/goptavares/intro-to-rl/blob/master/fig/sarsa_rewards.png?raw=true" height=200 width=320>

In [0]:
env = cliff_world()
# env = quentins_world()
# env = windy_cliff_grid()

params = default_params(environment=env)

# Decision-maker: choose parameter values.
params['epsilon'] = 0.01
params['alpha'] = 0.5
params['beta'] = 10
params['gamma'] = 0.8

# Choose a policy.
params['policy'] = 'epsilon_greedy'
# params['policy'] = 'softmax'

# Define number of episodes and maximum steps per episode.
n_episodes = 1000
max_steps = 1000

# Initialization.
# Start with uniform value function.
value = np.ones((env.n_states, env.n_actions))

# Run learning.
reward_sums, value = run_learning(value, params, n_episodes, max_steps)

fig = plot_state_action_values(env, value)
fig = plot_quiver_max_action(env, value)
fig = plot_heatmap_max_val(env, value)
fig = plot_rewards(n_episodes, reward_sums, average_range=10)